Import Libraries


In [132]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
import time
import re

In [133]:
headers = {"User Agent": "Mozilla/5.0"}

In [134]:
# Read player_extensions csv
url_extensions_df = pd.read_csv("data/player_extensions.csv")
# Convert Extension Column to list
url_extensions = url_extensions_df["Player Extension"].tolist()

In [135]:
# Create List with player links
player_links = []
def concat_urls(url_extension):
    for ext in url_extension:
        base_url = "https://www.basketball-reference.com/"
        player_url = ext
        full_url = base_url + player_url
        player_links.append(full_url)
concat_urls(url_extensions)

In [136]:
print(player_links[0])

https://www.basketball-reference.com//players/a/abdulka01.html


Scrape PPG

In [137]:
def scrape_ppg(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  # Raise an exception for bad HTTP status codes
        soup = BeautifulSoup(r.content, "html.parser")

        p1_div = soup.find("div", {"class": "p1"})
        if p1_div:
            ppg_value_element = p1_div.find_all("p")[3] 
            ppg_value = ppg_value_element.get_text()
            return ppg_value.strip()
        else:
            return "PPG information not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

# Test on First index of player_links

# print(scrape_ppg(player_links[0]))

Scrape Rebounds

In [138]:
def scrape_trb(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        p1_div = soup.find("div", {"class": "p1"})
        if p1_div:
            ppg_value_element = p1_div.find_all("p")[5] 
            ppg_value = ppg_value_element.get_text()
            return ppg_value.strip()
        else:
            return "PPG information not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

# print(scrape_trb(player_links[0]))

Scrape Assists

In [139]:
def scrape_ast(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        p1_div = soup.find("div", {"class": "p1"})
        if p1_div:
            ppg_value_element = p1_div.find_all("p")[7] 
            ppg_value = ppg_value_element.get_text()
            return ppg_value.strip()
        else:
            return "PPG information not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

# print(scrape_ast(player_links[0]))

Scrape MVP Count


In [140]:
def scrape_mvp_count(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        # Find the <ul> element with id "bling"
        bling_ul = soup.find('ul', {'id': 'bling'})
        
        if bling_ul:
            mvp_count = None
            for li in bling_ul.find_all('li'):
                a_tag = li.find('a')
                if a_tag and "MVP" in a_tag.text:
                    if "MVP" in a_tag.text and "Finals" not in a_tag.text:
                        mvp_count = re.search(r'\d+x', a_tag.text)
                        if mvp_count:
                            return mvp_count.group().replace('x', '')
            
            if mvp_count is None:
                return 0
        else:
            return "Bling section not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

print(scrape_mvp_count(player_links[2]))

2


Scrape Finals MVP

In [142]:
def scrape_fmvp_count(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        # Find the <ul> element with id "bling"
        bling_ul = soup.find('ul', {'id': 'bling'})

        if bling_ul:
            fmvp_count = 0
            fmvp_found = False
            for li in bling_ul.find_all('li'):
                a_tag = li.find('a')
                if a_tag and "Finals MVP" in a_tag.text:
                    fmvp_found = True
                    fmvp_text = a_tag.text.strip()
                    if 'Finals MVP' in fmvp_text:
                        if 'x' in fmvp_text:
                            count_match = re.search(r'\d+', fmvp_text)
                            if count_match:
                                fmvp_count += int(count_match.group())
                        else:
                            fmvp_count += 1
            if fmvp_found:
                return fmvp_count
            else:
                return 0  # Return 0 if no Finals MVP found
        else:
            return "Bling section not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

print(player_links[63])
# print(player_links[1])
# print(player_links[2])
print(scrape_fmvp_count(player_links[63]))
# print(scrape_fmvp_count(player_links[1]))
# print(scrape_fmvp_count(player_links[2]))
# print(player_links[17])
# print(scrape_fmvp_count(player_links[17])) # KD
# print(player_links[28])
# print(scrape_fmvp_count(player_links[28])) #MJ

https://www.basketball-reference.com//players/w/westje01.html
1


Scrape All NBA Team

In [143]:
def scrape_anbat_count(full_url):
    url = full_url
    try:
        time.sleep(7)
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")

        # Find the <ul> element with id "bling"
        bling_ul = soup.find('ul', {'id': 'bling'})

        if bling_ul:
            anbat_count = None
            for li in bling_ul.find_all('li'):
                a_tag = li.find('a')
                if a_tag and "All-NBA" in a_tag.text:
                    anbat_count = re.search(r'\d+x', a_tag.text)
                    if anbat_count:
                        return anbat_count.group().replace('x', '')

            if anbat_count is None:
                return 0
        else:
            return "Bling section not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

# Call the function with the player link
# print(scrape_anbat_count(player_links[0]))

Scrape NBA Championships Count

In [145]:
def scrape_nba_champ_count(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        # Find the <ul> element with id "bling"
        bling_ul = soup.find('ul', {'id': 'bling'})

        if bling_ul:
            nba_champ_count = 0
            nba_champ_found = False
            for li in bling_ul.find_all('li'):
                a_tag = li.find('a')
                if a_tag and "NBA Champ" in a_tag.text:
                    nba_champ_found = True
                    nba_champ_text = a_tag.text.strip()
                    if 'NBA Champ' in nba_champ_text:
                        if 'x' in nba_champ_text:
                            count_match = re.search(r'\d+', nba_champ_text)
                            if count_match:
                                nba_champ_count += int(count_match.group())
                        else:
                            nba_champ_count += 1
            if nba_champ_found:
                return nba_champ_count
            else:
                return 0  # Return 0 if no NBA Champ found
        else:
            return 0  # Return 0 if bling section not found
    except requests.RequestException as e:
        return f"An error occurred: {e}"
# print(player_links[16])
# print(scrape_nba_champ_count(player_links[16]))
# print(player_links[63])
# print(scrape_nba_champ_count(player_links[63]))
# print(player_links[2])
# print(scrape_nba_champ_count(player_links[2]))

All Defensive Teams Count

In [147]:
def scrape_all_def_team_count(full_url):
    url = full_url
    try:
        time.sleep(7)
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
                # Find the <ul> element with id "bling"
        bling_ul = soup.find('ul', {'id': 'bling'})

        if bling_ul:
            all_def_team_count = None
            for li in bling_ul.find_all('li'):
                a_tag = li.find('a')
                if a_tag and "All-Defensive" in a_tag.text:
                    all_def_team_count = re.search(r'\d+x', a_tag.text)
                    if all_def_team_count:
                        return all_def_team_count.group().replace('x', '')

            if all_def_team_count is None:
                return 0
        else:
            return "Bling section not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"
        
# print(scrape_all_def_team_count(player_links[0]))

Scrape PER Rating (Player Efficiency Rating)

In [148]:
def scrape_per_count(full_url):
    url = full_url
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  
        soup = BeautifulSoup(r.content, "html.parser")

        p1_div = soup.find("div", {"class": "p3"})
        if p1_div:
            ppg_value_element = p1_div.find_all("p")[1] 
            ppg_value = ppg_value_element.get_text()
            return ppg_value.strip()
        else:
            return "PER information not found"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

# print(scrape_per_count(player_links[0]))

Scrape Winshares/48

In [149]:
def scrape_ppg_list(player_link):
    ppg_list = []
    for player in player_link:
        ppg_list.append(scrape_ppg(player))
    return ppg_list
ppg_values = scrape_ppg_list(player_links)
print(ppg_values)

['24.6', '18.9', '23.5', '22.5', '22.8', '22.1', '24.8', '27.4', '21.8', '24.3', '12.4', '25.0', '24.7', '24.2', '16.1', '20.4', '27.3', '19.0', '17.8', '25.1', '19.2', '20.8', '21.0', '26.7', '27.1', '19.5', '17.7', '30.1', '12.6', '22.5', '20.1', '20.0', '25.1', '17.0', '25.0', '20.3', '24.2', '17.9', '23.1', '18.2', '18.8', '18.2', '19.0', '14.3', '20.7', '21.8', '23.7', '15.5', '17.4', '19.7', '16.1', '18.7', '25.7', '21.1', '15.1', '18.5', '17.8', '13.1', '19.2', '15.0', '10.8', '22.0', '13.3', '27.0', '16.5', '24.8', '17.6', '26.4', '18.7', '22.8', '25.1', '24.1', '20.7', '15.7', '19.5', '28.7', '16.7', '15.5', '9.8', '24.2', '17.1', '21.1', '30.1', '11.3', '23.6', '19.6', '7.0', '15.2', '16.1']


In [150]:
def scrape_trb_list(player_link):
    trb_list = []
    for player in player_link:
        trb_list.append(scrape_trb(player))
    return trb_list
trb_values = scrape_trb_list(player_links)
print(trb_values)

['11.2', '4.1', '9.8', '6.2', '8.6', '11.7', '6.7', '13.5', '4.1', '10.0', '3.2', '5.2', '4.7', '10.6', '11.0', '6.1', '7.0', '10.8', '10.0', '5.3', '5.0', '6.3', '12.5', '3.7', '7.5', '7.2', '4.9', '6.2', '6.3', '5.8', '10.1', '6.4', '4.2', '15.6', '10.1', '12.3', '4.2', '7.3', '13.4', '3.0', '3.0', '4.1', '9.2', '3.0', '7.5', '11.1', '10.9', '2.7', '4.5', '5.6', '6.4', '12.9', '7.5', '10.6', '22.5', '12.1', '3.9', '2.7', '3.6', '15.0', '14.0', '4.7', '10.5', '5.8', '4.7', '6.7', '5.1', '16.2', '12.9', '8.6', '5.3', '5.6', '9.8', '11.8', '3.5', '8.6', '4.3', '2.7', '10.3', '8.5', '8.5', '10.8', '22.9', '4.9', '4.0', '5.6', '4.8', '2.9', '2.2']


In [151]:
def scrape_ast_list(player_link):
    ast_list = []
    for player in player_link:
        ast_list.append(scrape_ast(player))
    return ast_list
ast_values = scrape_ast_list(player_links)
print(ast_values)

['3.6', '3.4', '4.9', '2.7', '2.3', '3.9', '4.9', '4.3', '4.3', '6.3', '3.4', '4.7', '6.4', '2.5', '2.9', '5.6', '4.4', '3.0', '3.7', '2.6', '4.0', '4.8', '1.8', '6.2', '7.4', '11.2', '2.5', '5.3', '8.7', '3.3', '3.1', '3.0', '6.7', '3.3', '3.6', '1.3', '5.4', '1.7', '2.8', '3.0', '3.9', '3.5', '1.6', '8.5', '2.4', '2.5', '2.5', '5.6', '9.3', '3.5', '5.2', '1.8', '9.5', '2.5', '4.3', '3.1', '3.0', '10.5', '9.3', '2.7', '3.9', '5.4', '3.4', '6.7', '6.7', '2.5', '3.0', '3.0', '1.8', '2.3', '2.6', '7.1', '4.2', '1.3', '2.3', '8.3', '3.1', '5.6', '1.0', '4.2', '1.1', '7.0', '4.4', '4.2', '5.7', '4.4', '2.1', '5.4', '4.5']


In [152]:
def scrape_mvp_list(player_link):
    mvp_list = []
    for player in player_link:
        mvp_list.append(scrape_mvp_count(player))
    return mvp_list
mvp_values = scrape_mvp_list(player_links)
print(mvp_values)
        

['6', 0, '2', 0, 0, 0, 0, 0, 0, '3', 0, '4', '2', 0, 0, 0, '2', '2', 0, 0, 0, 0, 0, '2', '3', '3', 0, '3', 0, 0, 0, 0, 0, 0, '2', '3', 0, 0, '2', 0, 0, 0, 0, '2', 0, 0, '3', 0, 0, 0, 0, '2', '3', 0, '5', 0, 0, 0, '2', 0, 0, 0, 0, 0, 0, 0, 0, '4', '2', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '4', 0, '3', '4', 0, 0, 0, 0, 0, 0]


In [153]:
def scrape_anbat(player_link):
    anbat_list = []
    for player in player_link:
        anbat_list.append(scrape_anbat_count(player))
    return anbat_list
anbat_values = scrape_anbat(player_links)
print(anbat_values)      

['15', '2', '8', '6', '4', '11', '6', '10', 0, '10', 0, '15', '10', '5', 0, '5', '11', '15', '9', '7', '7', '11', '6', '7', '20', '10', '3', '11', '6', '4', 0, '6', '7', '5', '14', '8', '4', 0, 0, '3', 0, '4', '5', '7', '12', '12', '14', '4', '11', '4', '7', '5', '11', '10', '11', '12', '7', '11', '5', 0, 0, '8', '2', '12', 0, '7', '2', '11', '5', '4', '7', '7', '5', '8', '2', '5', '2', '4', '3', '7', '2', '6', '10', 0, '3', '7', 0, '3', '3']


In [154]:
def scrape_all_def_team(player_link):
    all_def_team_list = []
    for player in player_link:
        all_def_team_list.append(scrape_all_def_team_count(player))
    return all_def_team_list
all_def_team_values = scrape_all_def_team(player_links)
print(all_def_team_values)

['11', 0, '5', 0, 0, 0, 0, 0, 0, '3', 0, '12', 0, '5', '6', 0, 0, '15', '12', 0, 0, '8', '2', 0, '6', 0, 0, '9', '9', 0, 0, '7', 0, 0, '4', '2', 0, '6', 0, 0, 0, 0, 0, 0, 0, '9', '3', 0, '9', 0, '10', 0, 0, '8', 0, 0, 0, '5', 0, '5', 0, '3', '2', '5', 0, 0, 0, 0, 0, 0, 0, 0, 0, '5', 0, 0, 0, 0, '6', 0, '2', 0, '2', '2', 0, 0, 0, '2', '5']


In [155]:
def scrape_per(player_link):
    per_list = []
    for player in player_link:
        per_list.append(scrape_per_count(player))
    return per_list
per_values = scrape_per(player_links)
print(per_values)

['24.6', '18.6', '25.5', '19.5', '19.8', '24.6', '21.0', '22.7', '18.2', '23.5', '12.2', '22.9', '23.6', '26.9', '15.5', '21.1', '25.0', '24.2', '22.7', '21.4', '15.7', '17.5', '17.7', '20.9', '27.0', '24.1', '18.7', '27.9', '17.9', '19.2', '21.7', '23.4', '22.3', '18.9', '23.9', '22.0', '18.4', '20.0', '27.1', '18.4', '17.2', '18.8', '23.0', '20.0', '22.4', '23.6', '26.4', '18.2', '23.9', '19.7', '18.6', '18.6', '23.2', '26.2', '18.9', '22.1', '18.3', '21.8', '18.1', '16.5', '16.0', '23.5', '20.0', '22.9', '16.8', '21.6', '17.7', '25.4', '18.6', '19.8', '21.4', '23.9', '20.9', '21.3', '15.9', '25.6', '18.6', '18.2', '17.2', '23.6', '21.2', '28.2', '26.2', '15.4', '22.4', '22.1', '13.4', '18.8', '15.3']


In [156]:
def scrape_fmvp(player_link):
    fmvp_list = []
    for player in player_link:
        fmvp_list.append(scrape_fmvp_count(player))
    return fmvp_list
fmvp_values = scrape_fmvp(player_links)
print(fmvp_values)

[2, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 0, 0, 0, 2, 3, 0, 0, 0, 1, 0, 0, 4, 3, 0, 6, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1]


In [157]:
def scrape_champ(player_link):
    champ_list = []
    for player in player_link:
        champ_list.append(scrape_nba_champ_count(player))
    return champ_list
champ_values = scrape_champ(player_links)
print(champ_values)

[6, 2, 1, 0, 1, 0, 1, 0, 0, 3, 2, 5, 4, 1, 2, 1, 2, 5, 1, 0, 1, 8, 1, 0, 4, 5, 10, 6, 1, 0, 0, 2, 0, 1, 0, 1, 0, 3, 5, 0, 1, 0, 0, 0, 1, 2, 4, 4, 0, 1, 6, 2, 1, 2, 11, 1, 4, 0, 2, 0, 1, 3, 2, 1, 0, 0, 3, 1, 2, 1, 0, 0, 0, 1, 4, 0, 0, 4, 0, 1, 1, 1, 2, 4, 1, 0, 7, 1, 2]


In [169]:
def scrape_ws48(full_url):
    url = full_url
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        time.sleep(7)  
        r = requests.get(url, headers=headers)
        r.raise_for_status()  # Raise an exception for bad HTTP status codes
        soup = BeautifulSoup(r.content, "html.parser")
        
        tables = soup.find_all("table")
        target_table = None
        for table in tables:
            headers = [header.get_text() for header in table.find_all("th")]
            if "Player" in headers and "WS/48" in headers:
                target_table = table
                break
        
        if target_table:
            rows = target_table.find_all("tr")[1:]  # Skip the header row

            data = []
            for row in rows:
                cells = row.find_all(["th", "td"])
                data.append([cell.get_text() for cell in cells])

            df = pd.DataFrame(data, columns=headers)
            return df
        else:
            return "Table not found on the page"
    except requests.RequestException as e:
        return f"An error occurred: {e}"

ws48 = "https://www.basketball-reference.com/leaders/ws_per_48_career.html"
ws48_df = scrape_ws48(ws48)
ws48_df.to_csv(r"C:\Users\Kai\PycharmProjects\NBA_Goat_Tracker\data\Player WS/48.csv", index=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2530628294.py, line 36)

In [158]:
player_names_series = url_extensions_df['Player Name']
ppg_series = pd.DataFrame(ppg_values, columns=['PPG'])
ast_series = pd.DataFrame(ast_values, columns=['AST'])
mvp_series = pd.DataFrame(mvp_values, columns=['RSMVP'])
anbat_series = pd.DataFrame(anbat_values, columns=['All NBA Team Selections'])  
f_mvp_series = pd.DataFrame(fmvp_values, columns=['FMVP'])
all_def_team_series = pd.DataFrame(all_def_team_values, columns=['All Defensive Teams'])
per_series = pd.DataFrame(per_values, columns=['PER'])
trb_series = pd.DataFrame(trb_values, columns=['TRB'])
champ_series = pd.DataFrame(champ_values, columns=["Championships"])
df = pd.concat([player_names_series,ppg_series,trb_series,ast_series, mvp_series, anbat_series,champ_series,f_mvp_series,all_def_team_series,per_series ], axis=1)
df

,Player Name,PPG,TRB,AST,RSMVP,All NBA Team Selections,Championships,FMVP,All Defensive Teams,PER
0,Kareem Abdul-Jabbar,24.6,11.2,3.6,6,15,6,2,11,24.6
1,Ray Allen,18.9,4.1,3.4,0,2,2,0,0,18.6
2,Giannis Antetokounmpo,23.5,9.8,4.9,2,8,1,1,5,25.5
3,Carmelo Anthony,22.5,6.2,2.7,0,6,0,0,0,19.5
4,Paul Arizin,22.8,8.6,2.3,0,4,1,0,0,19.8
...,...,...,...,...,...,...,...,...,...,...
84,Kyrie Irving,23.6,4.0,5.7,0,3,1,0,0,22.4
85,Tracy McGrady,19.6,5.6,4.4,0,7,0,0,0,22.1
86,Robert Horry,7.0,4.8,2.1,0,0,7,0,0,13.4
87,Chauncy Billups,15.2,2.9,5.4,0,3,1,1,2,18.8


Lets now save the df to a CSV File

In [ ]:
df.to_csv('data/NBA_Goat_Tracker_Data.csv')